##### Import of libraries

In [8]:
import numpy as np
from pettingzoo.classic import tictactoe_v3
from ctransformers import AutoModelForCausalLM
import os
import numpy as np
import re
import requests
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import random
from collections import deque

##### Stepst to create DeepLearning agent

Define the Neural Network Class (Q-Network) - simple MLP network

In [9]:
class QNetwork(nn.Module):
    def __init__(self, input_size=9, output_size=9):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


Define DQAgent class with PyTorch

In [ ]:

class DQAgent:
    def __init__(self, name, state_size, action_size, epsilon=0.1, lr=0.001, gamma=0.95):
        self.name = name
        self.state_size = state_size
        self.action_size = action_size
        self.epsilon = epsilon  # exploration rate
        self.gamma = gamma  # discount factor
        self.memory = deque(maxlen=2000)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = QNetwork(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.loss_fn = nn.MSELoss()

    
    def choose_action(self, state):
        valid_moves = np.where(state == 0)[0]

        if np.random.rand() < self.epsilon:
            # Exploration: Choose random valid move
            action = int(np.random.choice(valid_moves))
            print(f"[{self.name}] (Exploring) Chosen action: {action}")
        else:
            # Exploitation: Choose best move based on Q-values
            state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            with torch.no_grad():
                q_values = self.model(state_tensor).cpu().numpy().flatten()

            masked_q_values = np.full(q_values.shape, -np.inf)
            masked_q_values[valid_moves] = q_values[valid_moves]
            action = int(np.argmax(masked_q_values))
            print(f"[{self.name}] Valid moves: {valid_moves}")
            print(f"[{self.name}] Chosen action: {action}")

        return action

    def remember(self, state, action, reward):
        self.memory.append((state, action, reward))

    def replay(self, batch_size=32):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)
        states, actions, rewards = zip(*minibatch)

        states_tensor = torch.FloatTensor(states).to(self.device)
        actions_tensor = torch.LongTensor(actions).to(self.device)
        rewards_tensor = torch.FloatTensor(rewards).to(self.device)

        self.optimizer.zero_grad()
        predictions = self.model(states_tensor)
        predicted_q = predictions.gather(1, actions_tensor.unsqueeze(1)).squeeze()

        # Q-learning update (using immediate reward as this is episodic TicTacToe)
        loss = self.loss_fn(predicted_q, rewards_tensor)
        loss.backward()
        self.optimizer.step()

    # Function to store to save the DQ Agent model
    def save_model(self, path="dq_model.pth"):
        """Save model weights to disk."""
        torch.save(self.model.state_dict(), path)
        print(f"Model saved to {path}")

    # Function to load the trained DQ Agent model from disk for the evaluation phase
    def load_model(self, path="dq_model.pth"):
        """Load model weights from disk."""
        self.model.load_state_dict(torch.load(path))
        self.model.eval()  # Set model to evaluation mode
        print(f"Model loaded from {path}")


    

##### Steps to create LLM agent

In [11]:
# LLM API details (Modify if needed)
LLM_API_URL = "http://localhost:1234/v1/chat/completions"  # Change to your LM Studio API URL
MODEL_NAME ="Mistral-Nemo-Instruct-2407-GGUF"
#MODEL_NAME = "Phi-4-mini-instruct-GGUF"

Definition of the instruction

In [12]:
instruction = """
You are an expert Tic-Tac-Toe player. You must always follow the best strategy to win the game or block your opponent from winning.

Use this reasoning process:

1. Check if YOU can win on this turn. If yes, make that move.
2. If not, check if your OPPONENT could win on their next move. If yes, block it.
3. Otherwise, choose the best available strategic position (like center or corners).

Output ONLY the number of the chosen move (0–8) based on your decision.

Think step-by-step before giving the final number. But output only the number, without explanation.
"""

Building the prompt using information from the environment

In [13]:
def build_prompt(board_array, valid_moves, agent_name):
    board_str = ""
    cell_num = 0
    for row in board_array:
        row_str = ""
        for cell in row:
            if cell[0] == 1:
                row_str += "X"
            elif cell[1] == 1:
                row_str += "O"
            else:
                row_str += str(cell_num)
            cell_num += 1
        board_str += row_str + "\n"

    current_player = "X" if agent_name == "player_1" else "O"
    opponent = "O" if current_player == "X" else "X"

    prompt = f"""
    You are playing as {current_player}. Your opponent is {opponent}.

    This is the current Tic-Tac-Toe board. The numbers indicate empty positions:

    {board_str}

    Valid move positions: {valid_moves.tolist()}

    Think step-by-step:
    - Can you win this turn? Play that move.
    - Can your opponent win next turn? Block it.
    - Otherwise, choose the best long-term position.

    Now make your move by writing only the position number (0–8).
    """
    return prompt.strip()

Querying the LLM with the instruction and prompt

In [14]:
def query_llm(model, api_url, agent_name, instruction, prompt):
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": instruction},
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.0,
        "max_tokens": 8,
        "stop": ["\n"]
    }

    try:
        # Added debug prints to check the payload and API URL
        #print(f"\n[{agent_name}] Sending request to: {api_url}")
        #print(f"[{agent_name}] Payload:\n{json.dumps(payload, indent=2)}")

        response = requests.post(api_url, json=payload)
        response.raise_for_status()
        raw = response.json()

        # Added debug prints to check the raw response
        #print(f"[{agent_name}] Raw response:\n{json.dumps(raw, indent=2)}")

        return raw["choices"][0]["message"]["content"].strip()

    except Exception as e:
        print(f"[{agent_name}] LLM call failed: {e}")
        return ""

Parse LLM output, if LLM chose invalid move or if LLM move is not understood use fallback

In [15]:
def parse_llm_output(output, valid_moves, agent_name):
    match = re.search(r"\b(\d)\b", output.strip())
    if match:
        move = int(match.group(1))
        if move in valid_moves:
            return move
        else:
            print(f"[{agent_name}] LLM chose invalid move: {move}, not in {valid_moves}")
    else:
        print(f"[{agent_name}] LLM output not understood: '{output}'")

    fallback = int(np.random.choice(valid_moves))
    print(f"[{agent_name}] Fallback random move: {fallback}")
    return fallback

Define LLM Agent class with action method

In [16]:
class LLM_Agent:
    def __init__(self, name, model=MODEL_NAME, api_url="http://localhost:1234/v1/chat/completions"):
        self.name = name
        self.model = model
        self.api_url = api_url
    
    def get_action(self, observation, agent_name):
        board = observation["observation"].reshape(3, 3, 2)
        valid_moves = np.flatnonzero(observation["action_mask"])
        print(f"[{self.name}] valid moves: {valid_moves}\n")
        
        prompt = build_prompt(board, valid_moves, agent_name)
        response = query_llm(self.model, self.api_url, self.name, instruction, prompt)

        action = parse_llm_output(response, valid_moves, self.name)
        print(f"[{self.name}] Chosen action: {action}\n")

        return action

##### Steps to create the game using the two agents

In [17]:
# Use this to detect the winner 
def print_final_board(board):
    for row in board:
        row_str = ""
        for cell in row:
            if cell[0] == 1:
                row_str += "X"
            elif cell[1] == 1:
                row_str += "O"
            else:
                row_str += " "
        print(row_str)

# Use this function to detect the winner because the rewards from the environment are always 0
def detect_winner(board):
    board = board.argmax(axis=2)  # 0 for X, 1 for O, 2 = empty
    for player, label in [(0, "player_1"), (1, "player_2")]:
        for i in range(3):
            if all(board[i, :] == player): return label
            if all(board[:, i] == player): return label
        if all(np.diag(board) == player): return label
        if all(np.diag(np.fliplr(board)) == player): return label
    return None



Function for transforming the game state from the Tic-Tac-Toe environment into a format 
the DQAgent's neural network can understand and learn from

In [18]:
def preprocess_observation(obs):
    board = obs["observation"]
    board_processed = board[:,:,0] - board[:,:,1]
    return board_processed.flatten()

##### Running game once with no training DQ Agent

In [3]:
def run_game(render=False):
    render_mode = "human" if render else None
    env = tictactoe_v3.env(render_mode=render_mode)
    env.reset(seed=None)

    agent_1 = LLM_Agent(
        name="Mistral_X",
        model="Mistral-Nemo-Instruct-2407-GGUF",
        api_url="http://localhost:1234/v1/chat/completions"
    )

    agent_2 = DQAgent(
        name="DQAgent_O",
        state_size=9,
        action_size=9,
        epsilon=0.1
    )

    agents = {"player_1": agent_1, "player_2": agent_2}
    player_symbols = {"player_1": "X", "player_2": "O"}
    all_agents = env.possible_agents[:]
    move_history = []
    state_memory = []
    winner = None

    start_time = time.time()

    last_obs = None  # Save the last meaningful board state

    for agent_name in env.agent_iter():
        obs, reward, terminated, truncated, info = env.last()
        done = terminated or truncated

        if not done:
            last_obs = obs  # Save last non-terminal obs

            if agent_name == "player_1":
                action = agents[agent_name].get_action(obs, agent_name)
            else:
                state = preprocess_observation(obs)
                action = agents[agent_name].choose_action(state)
                state_memory.append((state, action))

            move_history.append((agent_name, action))
        else:
            action = None  # step(None) for done agent

        env.step(action)

    end_time = time.time()
    elapsed_time = end_time - start_time

    # Use last_obs to evaluate final board (non-terminal)
    if last_obs is not None:
        final_board = last_obs["observation"].reshape(3, 3, 2)
    else:
        final_obs = env.observe(all_agents[0])
        final_board = final_obs["observation"].reshape(3, 3, 2)

    winner = detect_winner(final_board)

    rewards = {agent: 0 for agent in all_agents}
    if winner:
        rewards[winner] = 1
        loser = [a for a in all_agents if a != winner][0]
        rewards[loser] = -1

    final_reward = rewards["player_2"]
    for state, action in state_memory:
        agent_2.remember(state, action, final_reward)
    agent_2.replay()

    print("🏁 Game Over!")
    print_final_board(final_board)
    print(f"⏱️ Duration: {elapsed_time:.2f} sec\nRewards: {rewards}")
    print(f"🏆 Winner: {agents[winner].name}" if winner else "🤝 Draw")

    for i, (agent, move) in enumerate(move_history):
        print(f"Turn {i+1}: {agent} → {move}")

    env.close()
    return rewards


In [54]:
if __name__ == "__main__":
    run_game(render=True)

[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] Valid moves: [0 1 3 4 5 6 7 8]
[DQAgent_O] Chosen action: 0
[Mistral_X] valid moves: [1 3 4 5 6 7 8]

[Mistral_X] Chosen action: 3

[DQAgent_O] Valid moves: [1 4 5 6 7 8]
[DQAgent_O] Chosen action: 1
[Mistral_X] valid moves: [4 5 6 7 8]

[Mistral_X] Chosen action: 5

[DQAgent_O] Valid moves: [4 6 7 8]
[DQAgent_O] Chosen action: 7
[Mistral_X] valid moves: [4 6 8]

[Mistral_X] Chosen action: 4

🏁 Game Over!
OOX
X X
 O 
⏱️ Duration: 17.12 sec
Rewards: {'player_1': 1, 'player_2': -1}
🏆 Winner: Mistral_X
Turn 1: player_1 → 2
Turn 2: player_2 → 0
Turn 3: player_1 → 3
Turn 4: player_2 → 1
Turn 5: player_1 → 5
Turn 6: player_2 → 7
Turn 7: player_1 → 4


##### Running the game 10 times with no training DQ agent

In [55]:
def run_multiple_episodes(n_episodes=10):
    total_rewards = {"player_1": 0, "player_2": 0}
    wins = {"player_1": 0, "player_2": 0}
    draws = 0

    for episode in range(1, n_episodes + 1):
        print(f"\n=== 🎮 Episode {episode} ===")
        render = (episode == n_episodes)  # Show only the final game
        rewards = run_game(render=render)

        for agent, reward in rewards.items():
            total_rewards[agent] += reward
            if reward == 1:
                wins[agent] += 1

        if all(r == 0 for r in rewards.values()):
            draws += 1

    # Win rate report
    print("\n==============================")
    print(f"📊 Results after {n_episodes} episodes:")
    for agent in total_rewards:
        print(f"🏆 {agent} → Wins: {wins[agent]} | Total Score: {total_rewards[agent]}")
    print(f"🤝 Draws: {draws}")
    print("==============================\n")

In [56]:
if __name__ == "__main__":
    run_multiple_episodes(n_episodes=10)


=== 🎮 Episode 1 ===
[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] Valid moves: [0 1 3 4 5 6 7 8]
[DQAgent_O] Chosen action: 8
[Mistral_X] valid moves: [0 1 3 4 5 6 7]

[Mistral_X] Chosen action: 5

[DQAgent_O] Valid moves: [0 1 3 4 6 7]
[DQAgent_O] Chosen action: 0
[Mistral_X] valid moves: [1 3 4 6 7]

[Mistral_X] Chosen action: 3

[DQAgent_O] Valid moves: [1 4 6 7]
[DQAgent_O] Chosen action: 7
[Mistral_X] valid moves: [1 4 6]

[Mistral_X] LLM chose invalid move: 2, not in [1 4 6]
[Mistral_X] Fallback random move: 1
[Mistral_X] Chosen action: 1

[DQAgent_O] Valid moves: [4 6]
[DQAgent_O] Chosen action: 6
🏁 Game Over!
XOO
O O
 XX
⏱️ Duration: 13.87 sec
Rewards: {'player_1': 1, 'player_2': -1}
🏆 Winner: Mistral_X
Turn 1: player_1 → 2
Turn 2: player_2 → 8
Turn 3: player_1 → 5
Turn 4: player_2 → 0
Turn 5: player_1 → 3
Turn 6: player_2 → 7
Turn 7: player_1 → 1
Turn 8: player_2 → 6

=== 🎮 Episode 2 ===
[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[

##### Running game 1000 times for training DQ Agent

In [ ]:
def run_game_with_agents(agent_1, agent_2, render=False):
    render_mode = "human" if render else None
    env = tictactoe_v3.env(render_mode=render_mode)
    env.reset(seed=None)

    agents = {"player_1": agent_1, "player_2": agent_2}
    all_agents = env.possible_agents[:]
    move_history = []
    state_memory = []

    last_obs = None
    winner = None

    for agent_name in env.agent_iter():
        obs, reward, terminated, truncated, info = env.last()
        done = terminated or truncated

        if not done:
            last_obs = obs

            if agent_name == "player_1":
                action = agent_1.get_action(obs, agent_name)
            else:
                state = preprocess_observation(obs)
                action = agent_2.choose_action(state)
                state_memory.append((state, action))
            
            move_history.append((agent_name, action))
        else:
            action = None

        env.step(action)

    # Final board state
    if last_obs is not None:
        final_board = last_obs["observation"].reshape(3, 3, 2)
    else:
        final_obs = env.observe(all_agents[0])
        final_board = final_obs["observation"].reshape(3, 3, 2)

    winner = detect_winner(final_board)

    rewards = {agent: 0 for agent in all_agents}
    if winner:
        rewards[winner] = 1
        loser = [a for a in all_agents if a != winner][0]
        rewards[loser] = -1

    # Train the DQAgent (only player_2)
    final_reward = rewards["player_2"]
    for state, action in state_memory:
        agent_2.remember(state, action, final_reward)
    agent_2.replay()

    # Always print full game output after each episode used for debugging purposes and to check the model performance
    print("🏁 Game Over!")
    print_final_board(final_board)
    print(f"Rewards: {rewards}")
    print(f"🏆 Winner: {agents[winner].name}" if winner else "🤝 Draw")
    for i, (agent, move) in enumerate(move_history):
        print(f"Turn {i+1}: {agent} → {move}")

    env.close()
    return rewards


In [ ]:
def run_multiple_episodes(n_episodes=1000, save_model_path="dq_model.pth"):
    agent_1 = LLM_Agent(
        name="Mistral_X",
        model="Mistral-Nemo-Instruct-2407-GGUF",
        api_url="http://localhost:1234/v1/chat/completions"
    )

    agent_2 = DQAgent(
        name="DQAgent_O",
        state_size=9,
        action_size=9,
        epsilon=1.0  # start with high exploration
    )

    wins = {"player_1": 0, "player_2": 0}
    draws = 0

    for episode in range(1, n_episodes + 1):
        print(f"\n=== 🎮 Episode {episode} ===")
        render = (episode == n_episodes)

        # Run one episode
        rewards = run_game_with_agents(agent_1, agent_2, render=render)

        # Tally results
        if rewards["player_1"] == 1:
            wins["player_1"] += 1
        elif rewards["player_2"] == 1:
            wins["player_2"] += 1
        else:
            draws += 1

        # Decay epsilon after each episode
        # This makes the agent explore a lot early on, and learn to exploit later, which is key for performance in DQN
        agent_2.epsilon = max(0.01, agent_2.epsilon * 0.995)

        # Show progress every 100 episodes (or on the first)
        if episode % 100 == 0 or episode == 1:
            print(f"After {episode} episodes:")
            print(f"    🏆 Wins — Player 1: {wins['player_1']}, Player 2: {wins['player_2']}")
            print(f"    🤝 Draws: {draws}")
    
    
    # Save model after training
    agent_2.save_model(save_model_path)

    print("\n=== Training Complete ===")
    print(f"🏆 Wins — Player 1: {wins['player_1']}, Player 2: {wins['player_2']}")
    print(f"🤝 Draws: {draws}")



In [ ]:
# Note output is used to analyze the models movements
if __name__ == "__main__":
    run_multiple_episodes(n_episodes=1000, save_model_path="dq_model.pth")

Training episodes:   0%|          | 0/1000 [00:00<?, ?it/s]

[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] Valid moves: [0 1 3 4 5 6 7 8]
[DQAgent_O] Chosen action: 3
[Mistral_X] valid moves: [0 1 4 5 6 7 8]

[Mistral_X] Chosen action: 5

[DQAgent_O] Valid moves: [0 1 4 6 7 8]
[DQAgent_O] Chosen action: 4
[Mistral_X] valid moves: [0 1 6 7 8]

[Mistral_X] Chosen action: 6

[DQAgent_O] Valid moves: [0 1 7 8]
[DQAgent_O] Chosen action: 8
[Mistral_X] valid moves: [0 1 7]

[Mistral_X] LLM chose invalid move: 3, not in [0 1 7]
[Mistral_X] Fallback random move: 0
[Mistral_X] Chosen action: 0

[DQAgent_O] Valid moves: [1 7]
[DQAgent_O] Chosen action: 1
[Mistral_X] valid moves: [7]



Training episodes:   0%|          | 1/1000 [00:17<4:45:56, 17.17s/it]

[Mistral_X] Chosen action: 7

🏁 Game Over!
XOX
OOX
X O
⏱️ Duration: 17.17 sec
Rewards: {'player_1': 0, 'player_2': 0}
🤝 Draw
Turn 1: player_1 → 2
Turn 2: player_2 → 3
Turn 3: player_1 → 5
Turn 4: player_2 → 4
Turn 5: player_1 → 6
Turn 6: player_2 → 8
Turn 7: player_1 → 0
Turn 8: player_2 → 1
Turn 9: player_1 → 7
[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] Valid moves: [0 1 3 4 5 6 7 8]
[DQAgent_O] Chosen action: 3
[Mistral_X] valid moves: [0 1 4 5 6 7 8]

[Mistral_X] Chosen action: 5

[DQAgent_O] Valid moves: [0 1 4 6 7 8]
[DQAgent_O] Chosen action: 4
[Mistral_X] valid moves: [0 1 6 7 8]

[Mistral_X] Chosen action: 6

[DQAgent_O] Valid moves: [0 1 7 8]
[DQAgent_O] Chosen action: 8
[Mistral_X] valid moves: [0 1 7]

[Mistral_X] LLM chose invalid move: 3, not in [0 1 7]
[Mistral_X] Fallback random move: 1
[Mistral_X] Chosen action: 1

[DQAgent_O] Valid moves: [0 7]
[DQAgent_O] Chosen action: 7
[Mistral_X] valid moves: [0]



Training episodes:   0%|          | 2/1000 [00:34<4:44:16, 17.09s/it]

...

Training episodes: 100%|█████████▉| 999/1000 [4:40:03<00:17, 17.37s/it]

[Mistral_X] Chosen action: 4

🏁 Game Over!
XOX
X O
OXO
⏱️ Duration: 17.43 sec
Rewards: {'player_1': 0, 'player_2': 0}
🤝 Draw
Turn 1: player_1 → 2
Turn 2: player_2 → 5
Turn 3: player_1 → 3
Turn 4: player_2 → 6
Turn 5: player_1 → 7
Turn 6: player_2 → 1
Turn 7: player_1 → 0
Turn 8: player_2 → 8
Turn 9: player_1 → 4
[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] Valid moves: [0 1 3 4 5 6 7 8]
[DQAgent_O] Chosen action: 5
[Mistral_X] valid moves: [0 1 3 4 6 7 8]

[Mistral_X] Chosen action: 3

[DQAgent_O] Valid moves: [0 1 4 6 7 8]
[DQAgent_O] Chosen action: 6
[Mistral_X] valid moves: [0 1 4 7 8]

[Mistral_X] LLM chose invalid move: 5, not in [0 1 4 7 8]
[Mistral_X] Fallback random move: 1
[Mistral_X] Chosen action: 1

[DQAgent_O] Valid moves: [0 4 7 8]
[DQAgent_O] Chosen action: 0
[Mistral_X] valid moves: [4 7 8]

[Mistral_X] Chosen action: 4

[DQAgent_O] Valid moves: [7 8]
[DQAgent_O] Chosen action: 7
[Mistral_X] valid moves: [8]



Training episodes: 100%|██████████| 1000/1000 [4:40:25<00:00, 16.83s/it]

[Mistral_X] LLM chose invalid move: 2, not in [8]
[Mistral_X] Fallback random move: 8
[Mistral_X] Chosen action: 8

🏁 Game Over!
OXX
XXO
OO 
⏱️ Duration: 21.92 sec
Rewards: {'player_1': 0, 'player_2': 0}
🤝 Draw
Turn 1: player_1 → 2
Turn 2: player_2 → 5
Turn 3: player_1 → 3
Turn 4: player_2 → 6
Turn 5: player_1 → 1
Turn 6: player_2 → 0
Turn 7: player_1 → 4
Turn 8: player_2 → 7
Turn 9: player_1 → 8


AttributeError: 'DQAgent' object has no attribute 'save_model'

In [24]:
if __name__ == "__main__":
    run_multiple_episodes(n_episodes=1000, save_model_path="dq_model.pth")


=== 🎮 Episode 1 ===
[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] (Exploring) Chosen action: 1
[Mistral_X] valid moves: [0 3 4 5 6 7 8]

[Mistral_X] Chosen action: 3

[DQAgent_O] (Exploring) Chosen action: 0
[Mistral_X] valid moves: [4 5 6 7 8]

[Mistral_X] Chosen action: 5

[DQAgent_O] (Exploring) Chosen action: 6
[Mistral_X] valid moves: [4 7 8]

[Mistral_X] Chosen action: 4

🏁 Game Over!
OOX
X X
O  
Rewards: {'player_1': 1, 'player_2': -1}
🏆 Winner: Mistral_X
Turn 1: player_1 → 2
Turn 2: player_2 → 1
Turn 3: player_1 → 3
Turn 4: player_2 → 0
Turn 5: player_1 → 5
Turn 6: player_2 → 6
Turn 7: player_1 → 4

=== 🎮 Episode 2 ===
[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] (Exploring) Chosen action: 7
[Mistral_X] valid moves: [0 1 3 4 5 6 8]

[Mistral_X] Chosen action: 3

[DQAgent_O] (Exploring) Chosen action: 1
[Mistral_X] valid moves: [0 4 5 6 8]

[Mistral_X] Chosen action: 5

[DQAgent_O] (Exploring) 

##### Step-by-Step: Evaluate a Trained DQAgent

Load the trained model

In [25]:
agent_O = DQAgent(
    name="DQAgent_O",
    state_size=9,
    action_size=9,
    epsilon=0.0  # Turn off exploration (only best actions)
)
agent_O.load_model("dq_model.pth")  # Path to your saved model

Model loaded from dq_model.pth


C:\Users\Habraken\AppData\Local\Temp\ipykernel_25244\52792537.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(path))


Create SLM agent

In [26]:
agent_X = LLM_Agent(
    name="Mistral_X",
    model="Mistral-Nemo-Instruct-2407-GGUF",
    api_url="http://localhost:1234/v1/chat/completions"
)

In [27]:
def evaluate_agent(agent_llm, agent_dq, n_games=50):
    wins = {"player_1": 0, "player_2": 0}
    draws = 0

    for episode in range(n_games):
        rewards = run_game_with_agents(agent_llm, agent_dq, render=False)

        if rewards["player_1"] == 1:
            wins["player_1"] += 1
        elif rewards["player_2"] == 1:
            wins["player_2"] += 1
        else:
            draws += 1

    print("\n=== Evaluation Results ===")
    print(f"🏆 Wins (LLM): {wins['player_1']}")
    print(f"🤖 Wins (DQAgent): {wins['player_2']}")
    print(f"🤝 Draws: {draws}")


In [28]:
if __name__ == "__main__":
    evaluate_agent(agent_X, agent_O, n_games=50)

[Mistral_X] valid moves: [0 1 2 3 4 5 6 7 8]

[Mistral_X] Chosen action: 2

[DQAgent_O] Valid moves: [0 1 3 4 5 6 7 8]
[DQAgent_O] Chosen action: 5
[Mistral_X] valid moves: [0 1 3 4 6 7 8]

[Mistral_X] Chosen action: 3

[DQAgent_O] Valid moves: [0 1 4 6 7 8]
[DQAgent_O] Chosen action: 6
[Mistral_X] valid moves: [0 1 4 7 8]

[Mistral_X] LLM chose invalid move: 5, not in [0 1 4 7 8]
[Mistral_X] Fallback random move: 4
[Mistral_X] Chosen action: 4

[DQAgent_O] Valid moves: [0 1 7 8]
[DQAgent_O] Chosen action: 0
[Mistral_X] valid moves: [1 7 8]

[Mistral_X] LLM chose invalid move: 3, not in [1 7 8]
[Mistral_X] Fallback random move: 7
[Mistral_X] Chosen action: 7

[DQAgent_O] Valid moves: [1 8]
[DQAgent_O] Chosen action: 1
[Mistral_X] valid moves: [8]

[Mistral_X] LLM chose invalid move: 2, not in [8]
[Mistral_X] Fallback random move: 8
[Mistral_X] Chosen action: 8

🏁 Game Over!
OOX
XXO
OX 
Rewards: {'player_1': 0, 'player_2': 0}
🤝 Draw
Turn 1: player_1 → 2
Turn 2: player_2 → 5
Turn 3: play